In [ ]:
!pip install langgraph langchain_core langchain_groq langchain_huggingface langchain_chroma

In [ ]:
from typing import Annotated,List,Literal
from typing_extensions import TypedDict
from langgraph.graph import START,END,StateGraph
from langgraph.prebuilt import ToolNode,tools_condition
from langgraph.graph.message  import add_messages
from langchain.chat_models import init_chat_model
from langchain_huggingface import HuggingFaceEmbeddings
from google.colab import userdata,drive
from langgraph.graph.message import add_messages
from langchain_chroma import Chroma
from langchain_core.tools import tool
drive.mount('/content/drive')
import os
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
db_persist_directory = "/content/drive/MyDrive/chroma_db_let_us_C"
drive_base_path='/content/drive/MyDrive'
hf_models_folder = 'HuggingFace_Models'
model_name_to_save = 'all-MiniLM-L6-v2'
hf_model_load_path = os.path.join(drive_base_path, hf_models_folder, model_name_to_save)
embedding_instance=HuggingFaceEmbeddings(model_name=model_name_to_save)

In [ ]:
@tool
def perform_math_operations(a:float , b:float, maths_operation:str) ->float:
  "Use this for functions for maths operations such as add, sub, multiply, divide and ensure you are foloowing BODMAS rule"
  if 'add' in maths_operation:
      return a+b
  elif 'sub' in maths_operation:
    return a-b
  elif 'multiply' in maths_operation:
    return a*b
  elif 'divide' in maths_operation:
    return a//b
  else:
    return None

In [ ]:
#llm call
llm= init_chat_model(model='gemma2-9b-it',model_provider='groq')

avail_tools=[perform_math_operations]

llm_with_tool=llm.bind_tools(avail_tools)

#retreiver
vector_db=Chroma(embedding_function=embedding_instance,persist_directory=db_persist_directory).as_retriever(kwargs={'k':5})

In [ ]:
class State(TypedDict):
  messages: Annotated[List,add_messages]

def llm_call(state:State):
  return {'messages':llm_with_tool.invoke(state['messages'])}

In [ ]:
graph=StateGraph(State)
graph.add_node('tool_llm',llm_call)
graph.add_node('tools',ToolNode(avail_tools))

graph.add_edge(START,'tool_llm')
graph.add_conditional_edges('tool_llm',tools_condition)
graph.add_edge('tools','tool_llm')

builder=graph.compile()


In [ ]:
message=builder.invoke({'messages':'find solution for this 5 + 2 + 4 - 10* 6? Ensure you are using BODMAS rule'})

In [ ]:
for m in message['messages']:
  m.pretty_print()

================================ Human Message =================================

find solution for this 5 + 2 + 4 - 10* 6? Ensure you are using BODMAS rule
================================== Ai Message ==================================
Tool Calls:
  perform_math_operations (9erj45r1y)
 Call ID: 9erj45r1y
  Args:
    a: 5
    b: 2
    maths_operation: +
================================= Tool Message =================================
Name: perform_math_operations

null
================================== Ai Message ==================================
Tool Calls:
  perform_math_operations (8yvtp67nt)
 Call ID: 8yvtp67nt
  Args:
    a: 2
    b: 4
    maths_operation: +
================================= Tool Message =================================
Name: perform_math_operations

null
================================== Ai Message ==================================
Tool Calls:
  perform_math_operations (01fmbpdsp)
 Call ID: 01fmbpdsp
  Args:
    a: 2
    b: 4
    maths_operation: +
=========

In [ ]:
async for events in builder.astream_events({'messages':'What is add of 5 and 2? '},stream_mode='values'):
  print(events)

{'event': 'on_chain_start', 'data': {'input': {'messages': 'What is add of 5 and 2? '}}, 'name': 'LangGraph', 'tags': [], 'run_id': 'd52b636a-5492-4d02-bb50-29bc92eb2827', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': 'd52b636a-5492-4d02-bb50-29bc92eb2827', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'chunk': {'messages': [HumanMessage(content='What is add of 5 and 2? ', additional_kwargs={}, response_metadata={}, id='ffd9e3fc-62a6-4219-a960-51fa41b2386f')]}}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='What is add of 5 and 2? ', additional_kwargs={}, response_metadata={}, id='ffd9e3fc-62a6-4219-a960-51fa41b2386f')]}}, 'name': 'tool_llm', 'tags': ['graph:step:1'], 'run_id': '04c242c3-80b6-4f4a-95db-8b22dbb1a8ce', 'metadata': {'langgraph_step': 1, 'langgraph_node': 'tool_llm', 'langgraph_triggers': ('branch:to:tool_llm',), 'langgraph_path': ('__pregel_pull', 'tool_llm'), 'langgraph_checkpoin

In [ ]:
async for events in builder.astream_events({'messages':'What is add of 5 and 2 and 18? '},stream_mode='values'):

    print(events)

{'event': 'on_chain_start', 'data': {'input': {'messages': 'What is add of 5 and 2 and 18? '}}, 'name': 'LangGraph', 'tags': [], 'run_id': 'b4867688-1d6f-4ba2-8d35-e596c0176e48', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': 'b4867688-1d6f-4ba2-8d35-e596c0176e48', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'chunk': {'messages': [HumanMessage(content='What is add of 5 and 2 and 18? ', additional_kwargs={}, response_metadata={}, id='96962f31-6274-40fe-85a1-3e616a8ef2eb')]}}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='What is add of 5 and 2 and 18? ', additional_kwargs={}, response_metadata={}, id='96962f31-6274-40fe-85a1-3e616a8ef2eb')]}}, 'name': 'tool_llm', 'tags': ['graph:step:1'], 'run_id': 'e696cd99-a33b-4894-b750-abce6e0cce9d', 'metadata': {'langgraph_step': 1, 'langgraph_node': 'tool_llm', 'langgraph_triggers': ('branch:to:tool_llm',), 'langgraph_path': ('__pregel_pull', 'tool_llm'),